In [9]:
#save_to_csv has the sparse matrix generation step
#  NOTE that the transition matrix is generated as normal (an ever increasing set of matrices)
#   while the output is that of sparse matrix.
#  Keep in mind that the 'row' count starts at order 0 is continues through the other orders
#   for this code. (for instance order 0's 1st row is 0, order 1's first row is 1 and 
#   order 2's first row is (number of characters)+1)
from typing import List
import numpy as np
import csv

#Part of Russell's code
def build_alphabet_from_dataset(dataset : List[List[str]]) -> List[str]:
    """Iterate through the dataset and build an alphabet of unique items."""
    alphabet = set()
    ordered_alphabet = []
    for row in dataset:
        for item in row:
            if item not in alphabet:
                ordered_alphabet.append(item)
                alphabet.add(item)
    return ordered_alphabet
#Russell's build_transition_matrix
def build_transition_matrix(
    dataset : List[List[str]],
    order : int,
    alphabet : List[str] = None
):
    """Build a set of transition matrices for a given dataset and order."""

    if alphabet is None:
        alphabet = build_alphabet_from_dataset(dataset)

    alphabet_length = len(alphabet)
    occurrence_mats = [
        np.zeros((alphabet_length,) * (i+1), dtype=np.uint16)
        for i in range(order + 1)
    ]

    p_starting_symbol = np.zeros(alphabet_length, dtype=np.uint16)
    n = np.zeros(order + 1, dtype=np.uint32)

    for cur_sequence in dataset:
        for cur_item_index in range(len(cur_sequence)):
            cur_item = cur_sequence[cur_item_index]
            cur_item_alphabet_index = alphabet.index(cur_item)

            if cur_item_index == 0:
                p_starting_symbol[cur_item_alphabet_index] += 1

            co_occuring_indexes = []
            for cur_order in range(0, order + 1):
                if len(cur_sequence) <= cur_item_index + cur_order:
                    continue

                n[cur_order] += 1
                next_order_item = cur_sequence[cur_item_index + cur_order]
                next_order_syllables_alphabet_index = alphabet.index(next_order_item)

                co_occuring_indexes.append(next_order_syllables_alphabet_index)
                occurrence_mats[cur_order][tuple(co_occuring_indexes)] += 1

    return {
        "occurrence_mats": occurrence_mats,
        "p_starting_symbol": p_starting_symbol,
        "alphabet": alphabet,
        "N": n
    }

#the function we call to run the program
def build_transition_matrix_order_4(dataset):
    order = 4
    result = build_transition_matrix(dataset, order)
    save_to_csv(result)

#the function to output the transition matrix as separate sparse matrices in separate
#  CSVs. additionally, it outputs the p_starting_symbol and the N_counts as their own CSV.
# NOTE: This is a partial solution and there's more to work out (like starting row count
#  for each of the individual CSVs at 0 instead of a continual count requiring the user
#  to perform calculations on how many rows come before)
def save_to_csv(result):
    alphabet = result["alphabet"]
    occurrence_mats = result["occurrence_mats"]
    p_starting_symbol = result["p_starting_symbol"]
    n = result["N"]

    # Save each occurrence matrix to a separate CSV file
    for i, mat in enumerate(occurrence_mats):
        # Note that the filename is not setup to change for
        #  different datasets so the program will overwrite
        #  the existing CSV file
        #Creates different CSV files for each order (order = i)
        filename = 'pre2ndhalf_occurrence_matrix_order_{}.csv'.format(i)
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if mat.ndim == 1:
                writer.writerow(mat)
            elif mat.ndim == 2:
                writer.writerows(mat)
            else:
                # Flatten and label higher-order matrices
                writer.writerow(['row','col','value'])
                for index, row in enumerate(mat.reshape(-1, mat.shape[-1])):

                    for col in range(len(list(row))):

                        if row[col] > 0:
                            #writes as 'row','col','value' only if value > 0
                            writer.writerow([index,col,row[col]])
    # Save p_starting_symbol as CSV
    # Note that the filename is not setup to change for
    #  different datasets so the program will overwrite
    #  the existing CSV file
    with open('p_starting_symbol.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Symbol"] + alphabet)
        writer.writerow(["Probability"] + list(p_starting_symbol))
    
    # Save N vector as CSV
    # Note that the filename is not setup to change for
    #  different datasets so the program will overwrite
    #  the existing CSV file
    with open('N_counts.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Order"] + [f'Order_{i}' for i in range(len(n))])
        writer.writerow(["Count"] + list(n))

filename = 'Presurgery-songs_second-half_2D-syllables'

with open(f'{filename}.csv', newline='', encoding='UTF8') as csvfile:
    reader = list(csv.reader(csvfile))
    build_transition_matrix_order_4(reader)